In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import requests
import sys
import time
import os
# Load the page
def pull_bref():
    frames = []
    for year in range(1997,2001):
        url = "https://www.basketball-reference.com/leagues/NBA_"+str(year)+"_totals.html"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the specific table (replace 'table_id' with the actual table id or class)
        table = soup.find('table')
        
        
        # Get all rows from the table body
        rows = table.find('tbody').find_all('tr')
        row=rows[0]
        print(row)
        print(rows[1])
        
        # Iterate over each row and extract links from the desired column
        links = []
        for row in rows:
            # Get the specific column (replace 'column_index' with the actual index)
            cell = row.find_all('td', {'class' : 'left'})
            if len(cell)>0:
                row = cell[0]
                print(row)
                player_name=row.a.text
    
                player_href = "https://www.basketball-reference.com"+row.a['href']
                player_year=year
        
                data=[player_name,player_href,player_year]
        
                links.append(data)
            
            
                # Find all anchor tags in the column
            
               
            
            # Output the list of links
            year_data =pd.DataFrame(data=links,columns=['player','url','year'])
            print(year)
        frames.append(year_data)
    return pd.concat(frames)
                                
#old_index = pull_bref()
old_index = pd.read_csv('old_index.csv')

frames = []
if os.path.exists('on_off.csv'):
    on_off = pd.read_csv('on_off.csv')
    
    on_off['tempkey'] = on_off['player']+on_off['year'].astype(str)
    
    old_index['tempkey'] = old_index['player']+old_index['year'].astype(str)
    tempkeys = on_off['tempkey'].tolist()

    old_index = old_index[~old_index.tempkey.isin(tempkeys)]

    frames.append(on_off)
    print(len(old_index))
count = 0
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',
    }
try:
    for index,row in old_index.iterrows():
        url = row['url']
        year = str(row['year'])
        url =url.split('.html')[0]
        
        url= url +'/on-off/'
        url = url+year
        dfs = pd.read_html(url,storage_options=headers)
        df=dfs[0]
        df.columns = df.columns.droplevel()
        new_columns = ['Split',
         'Tm',
         'MP',
         'eFG%',
         'ORB%',
         'DRB%',
         'TRB%',
         'AST%',
         'STL%',
         'BLK%',
         'TOV%',
         'ORtg',
         'eFG%_opp',
         'ORB%_opp',
         'DRB%_opp',
         'TRB%_opp',
         'AST%_opp',
         'STL%_opp',
         'BLK%_opp',
         'TOV%_opp',
         'ORtg_opp',
         'eFG%_diff',
         'ORB%_diff',
         'DRB%_diff',
         'TRB%_diff',
         'AST%_diff',
         'STL%_diff',
         'BLK%_diff',
         'TOV%_diff',
         'ORtg_diff']
        df.columns=new_columns
        df['year'] = int(year)
        df['player']=row['player']
   
        frames.append(df)
        time.sleep(2.5)
        count+=1
        if count %50==0:
            on_off = pd.concat(frames)
            on_off.to_csv('on_off.csv',index=False)
            print(count)
            
    on_off = pd.concat(frames)
    on_off.to_csv('on_off.csv',index=False)

                         
except Exception as e:
    print('failed')
    print(e)
    on_off = pd.concat(frames)
    on_off.to_csv('on_off.csv',index=False)


0


In [4]:
original_columns = ['Split', 'Tm', 'MP', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
                    'BLK%', 'TOV%', 'ORtg', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
                    'BLK%', 'TOV%', 'ORtg', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
                    'BLK%', 'TOV%', 'ORtg']

new_columns = []
occurrence_count = {}

for col in original_columns:
    if col not in occurrence_count:
        occurrence_count[col] = 1
        new_columns.append(col)
    else:
        occurrence_count[col] += 1
        if occurrence_count[col] == 2:
            new_columns.append(f'{col}_opp')
        elif occurrence_count[col] == 3:
            new_columns.append(f'{col}_diff')
        else:
            # Handle additional occurrences beyond the third if necessary
            new_columns.append(f'{col}_{occurrence_count[col]}')

new_columns

['Split',
 'Tm',
 'MP',
 'eFG%',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'ORtg',
 'eFG%_opp',
 'ORB%_opp',
 'DRB%_opp',
 'TRB%_opp',
 'AST%_opp',
 'STL%_opp',
 'BLK%_opp',
 'TOV%_opp',
 'ORtg_opp',
 'eFG%_diff',
 'ORB%_diff',
 'DRB%_diff',
 'TRB%_diff',
 'AST%_diff',
 'STL%_diff',
 'BLK%_diff',
 'TOV%_diff',
 'ORtg_diff']